In [6]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Number of variables (adjust as needed for a test)
num_m = 5  # for i = 1, ..., m
num_k = 4  # for l = 0, ..., k-1

# Initialize random data for d, f, β, B, C, α
np.random.seed(42)  # for reproducibility
d = np.random.rand(num_m)       # Random demand for each i
f = np.random.rand(num_m)       # Random costs for each i
beta = np.random.rand(num_k, num_m)  # Random coefficients for constraints
B = 10.0                         # Capacity constraint
C = np.random.rand(num_m) * 5    # Upper bound constraints
alpha = np.random.rand()         # Random value for alpha

# Initialize iteration values
L = -float("inf")
U = float("inf")
k = 1

# Create Gurobi Model for Lower Bound
model = gp.Model("robust_lower_bound_model")

# Define variables for the master problem
y = model.addVars(num_m, vtype=GRB.CONTINUOUS, lb=0, name="y")
r = model.addVars(num_m, vtype=GRB.BINARY, name="r")
alpha_var = model.addVar(vtype=GRB.CONTINUOUS, name="alpha")

# Define auxiliary variables for the upper bound calculation
u = model.addVars(num_m, vtype=GRB.CONTINUOUS, name="u")
v = model.addVars(num_k, vtype=GRB.CONTINUOUS, name="v")
z = model.addVars(num_k, vtype=GRB.CONTINUOUS, name="z")

# Objective function for the master problem
objective = gp.quicksum(d[i] * y[i] for i in range(num_m)) + \
            gp.quicksum(f[i] * r[i] for i in range(num_m)) + alpha_var
model.setObjective(objective, GRB.MINIMIZE)

# Demand constraint
model.addConstr(gp.quicksum(y[i] for i in range(num_m)) >= B, name="demand_constraint")

# Upper bound constraint
for i in range(num_m):
    model.addConstr(y[i] <= C[i] * r[i], name=f"upper_bound_constraint_{i}")

# Robust constraint for each l
for l in range(num_k):
    model.addConstr(
        alpha_var >= -gp.quicksum(y[i] * u[i] for i in range(num_m)) +
                    gp.quicksum(beta[l][i] * v[l] for i in range(num_m)) +
                    gp.quicksum(beta[l][i] * z[l] for i in range(num_m)),
        name=f"robust_constraint_{l}"
    )

# Iterative process to update L and U
while True:
    # Solve the lower bound model (master problem)
    model.optimize()
    
    if model.status == GRB.OPTIMAL:
        # Retrieve solution for lower bound variables
        y_values = model.getAttr("X", y)
        r_values = model.getAttr("X", r)
        alpha_val = alpha_var.X
        u_values = model.getAttr("X", u)
        v_values = model.getAttr("X", v)
        z_values = model.getAttr("X", z)
        
        # Update lower bound L
        L = sum(d[i] * y_values[i] for i in range(num_m)) + \
            sum(f[i] * r_values[i] for i in range(num_m)) + alpha_val
        print(f"Iteration {k}: Lower Bound L = {L}")
        
        # Upper Bound Calculation
        upper_bound = sum(d[i] * y_values[i] for i in range(num_m)) + \
                      sum(f[i] * r_values[i] for i in range(num_m)) - \
                      sum(y_values[i] * u_values[i] for i in range(num_m)) + \
                      sum(beta[j][i] * v_values[j] for j in range(num_k)) + \
                      sum(beta[j][i] * z_values[j] for j in range(num_k))
        
        # Update U if the calculated upper bound is better
        U = min(U, upper_bound)
        print(f"Iteration {k}: Upper Bound U = {U}")
        
        # Check stopping condition
        if U == L:
            print(f"Optimal solution found with L = {L} and U = {U}")
            print("Optimal values for y:", y_values)
            print("Optimal values for r:", r_values)
            print("Optimal values for u:", u_values)
            print("Optimal values for v:", v_values)
            print("Optimal values for z:", z_values)
            break
        
        # Step 3: Add the new constraint to the master problem
        model.addConstr(
            alpha_var >= -sum(y[i] * u_values[i] for i in range(num_m)) +
                        sum(beta[j][i] * v_values[j] for j in range(num_k)) +
                        sum(beta[j][i] * z_values[j] for j in range(num_k)),
            name=f"robust_constraint_{k}"
        )
        
        # Update iteration count
        k += 1
    else:
        print("Optimization did not find an optimal solution.")
        break

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B83)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 24 columns and 15 nonzeros
Model fingerprint: 0x36ab995b
Model has 4 quadratic constraints
Variable types: 19 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [3e-01, 5e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+00]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 2 rows and 17 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 10 nonzeros
Variable types: 4 continuous, 3 integer (3 binary)
Found heuristic solution: objective 4.6338211

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T